# Генерация Текста

Самой сильной генеративной моделью на сегодняшний день является GPT-3. Мы построим модель с такой же архитектурой, но меньшего масштаба.

Генерировать текст мы будем с помощью языковой модели, сэмплируя токен за токеном.

In [ ]:
!pip install torch==1.7 torchtext==0.8 tokenizers

     |████████████████████████████████| 6.9MB 7.4MB/s 
     |████████████████████████████████| 3.2MB 39.1MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


Определим нашу модель. Как и модели семейства GPT, это просто несколько слоёв Transformer Decoder-а.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class Model(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_heads, n_layers, dropout):
        super(Model, self).__init__()

        self.vocab_size = vocab_size
        self.emb = nn.Embedding(vocab_size, hidden_size)

        self.pos_emb = PositionalEncoding(hidden_size)
 
        layer = TransformerEncoderLayer(hidden_size, n_heads, hidden_size, dropout)

        self.layers = TransformerEncoder(layer, n_layers)

        self.out = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, padding_mask):
        x_len = x.size(1)

        x = self.pos_emb(self.emb(x) * math.sqrt(self.vocab_size))

        attn_mask = nn.Transformer.generate_square_subsequent_mask(None, x_len).to(device)

        out = self.layers(x.transpose(0, 1), attn_mask, padding_mask).transpose(0, 1)

        out = self.out(out)

        return out

class PositionalEncoding(nn.Module):
    def __init__(self, hidden_size, dropout=0.1, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, hidden_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_size, 2).float() * (-math.log(10000.0) / hidden_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

Можно заметить, однако, что в коде выше используется модуль из pytorch, который называется TransfomerEncoder. Существует некоторая путаница, что называть Transformer Decoder-ом. В оргинальной статье https://arxiv.org/abs/1706.03762 декодер имеет два блока внимания, self-attention, и attention, который "смотрит" на выходы энкодера. При этом в GPT используется только self-attention. Отличие от энкодера в авторегрессионной маске аттеншена, которая позволяет смотреть только на предыдущие токены. 

Для данных будем использовать датасет, состоящий из стихотворений русских классиков. Для токенов обучим Byte-level BPE из библиотеки tokenizers c достаточно большим размером словаря. 

In [ ]:
from torchtext.utils import download_from_url



train_filename = download_from_url('https://raw.githubusercontent.com/sberbank-ai/classic-ai/master/data/classic_poems.json')

NameError: ignored

In [ ]:
import json

len(json.load(open(train_filename, encoding='utf-8')))

2496

In [ ]:
import torch

import re
from tokenizers import ByteLevelBPETokenizer
import json

def get_data_poems(text_filename, vocab_size):
  tokenizer = ByteLevelBPETokenizer(dropout=0.1, lowercase=True)

  poems = json.load(open(train_filename, encoding='utf-8'))

  poems = [poem['content'] for poem in poems]

  tokenizer.train_from_iterator(poems, vocab_size=vocab_size)

  tokenizer.add_special_tokens(["[SOS]", "[EOS]", "[PAD]"])

  SOS_id = tokenizer.token_to_id("[SOS]")
  EOS_id = tokenizer.token_to_id("[EOS]")

  nl_id = tokenizer.encode("\n").ids[0]

  poem_ids = []
  for poem in poems:
    lines = poem.split("\n")

    chunk = []
    for line in lines:
      line_ids = tokenizer.encode(line).ids

      if len(chunk) + len(line_ids) < 64:
        chunk.extend([nl_id] + line_ids)

      elif not chunk:
        continue

      else:
        poem_ids.append([SOS_id] + chunk + [EOS_id])

        if len(line_ids) < 64:
          chunk = line_ids
        else:
          chunk = []
    
    if not chunk:
      poem_ids.append([SOS_id] + chunk + [EOS_id])

  return LMDataset(poem_ids), tokenizer


class LMDataset(torch.utils.data.Dataset):
    def __init__(self, sentence_ids):
        super(LMDataset).__init__()
        self.data = sentence_ids

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


def collate_fn_lm(PAD_id, samples):
    batch_size = len(samples)

    max_len = max(len(sample) for sample in samples)

    src_tensor = torch.ones((batch_size, max_len), dtype=torch.long) * PAD_id

    lengths = []
    for (batch_id, s) in enumerate(samples):
        length = len(s)

        src_tensor[batch_id][:length] = torch.tensor(s)

        lengths.append(length)

    return src_tensor, torch.tensor(lengths)


In [ ]:
dataset, tokenizer = list(get_data_poems(train_filename, 8192))

SOS_id = tokenizer.token_to_id("[SOS]")
EOS_id = tokenizer.token_to_id("[EOS]")
PAD_id = tokenizer.token_to_id("[PAD]")

In [ ]:
print(f"{len(dataset)} стихов")
print("Пример:\n")

print(tokenizer.decode(dataset[220]))

9870 стихов
Пример:


вот зеркало мое – прими его, киприда!
богиня красоты прекрасна будет ввек,
седого времени не страшна ей обида:
она – не смертный человек;
но я, покорствуя судьбине,
не в силах зреть себя в прозрачности стекла


In [ ]:
from torch.utils.data import DataLoader
from functools import partial

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = tokenizer.get_vocab_size()
hidden_size = 512
n_layers = 4
n_heads = 4
dropout = 0.1

batch_size = 128
epochs = 32

model = Model(vocab_size, hidden_size, n_heads, n_layers, dropout).to(device)

data_loader = DataLoader(
    dataset
    , batch_size=batch_size
    , shuffle=True
    , collate_fn=partial(collate_fn_lm, PAD_id)
)


criterion = nn.CrossEntropyLoss(reduction='none')
lr = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
from tqdm import tqdm
def train(model, data_loader, epochs):
    for epoch in range(1, epochs+1):
      total_loss = 0.0
      for batch, _ in tqdm(data_loader):
          batch = batch.to(device)
          src = batch[:, :-1]
          tar = batch[:, 1:]

          optimizer.zero_grad()

          padding_mask = (src == PAD_id)

          out = model(src, padding_mask)

          loss = criterion(out.transpose(-2, -1), tar)[src != PAD_id].mean()

          total_loss += loss.item()

          loss.backward()
          grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

          optimizer.step()

      print(f'epoch {epoch:3d},  loss {total_loss / len(data_loader):.2f}')
    
    return model

In [ ]:
model = train(model, data_loader, epochs)

model.eval()

print("OK")

  1%|▏         | 1/78 [00:00<00:11,  6.82it/s]

epoch   1,  loss 6.67


  1%|▏         | 1/78 [00:00<00:11,  6.72it/s]

epoch   2,  loss 6.14


  1%|▏         | 1/78 [00:00<00:11,  6.77it/s]

...

epoch  30,  loss 3.69


  1%|▏         | 1/78 [00:00<00:11,  6.82it/s]

epoch  31,  loss 3.63


100%|██████████| 78/78 [00:11<00:00,  6.66it/s]

epoch  32,  loss 3.59
OK


Языковая модель обучена. Как теперь генерировать новые тексты из неё? Раз сеть выдаёт распределение на токенах на каждом шаге, то можно сэмплировать новый токен в соответствие с этим распределением:

In [ ]:
def sample_generate(model, ids, max_len, EOS_id):
    for j in range(len(ids), max_len):
      x = torch.tensor(ids).unsqueeze(0).to(device)

      x_len = x.size(1)

      x = model.pos_emb(model.emb(x))

      attn_mask = nn.Transformer.generate_square_subsequent_mask(None, x_len).to(device)

      out = model.layers(x.transpose(0, 1), attn_mask).transpose(0, 1)

      out = model.out(out)

      dist = torch.distributions.categorical.Categorical(logits=out[0][x_len-1])

      next_id = dist.sample().item()

      if next_id == EOS_id:
        break

      ids.append(next_id)

    return ids

In [ ]:
model.eval()

start_ids = [SOS_id]

sample_ids = sample_generate(model, start_ids, 64, EOS_id)

sent = tokenizer.decode(sample_ids[1:])

print(sent)


вых стая проду прежних, кто твор счастьем языком
ище под всеми вдохновенный подними свою прет,
чтоб сердца об мир: щи полный прябольно
звленок кресты, поэты в феным – тряс все:
твоей, маске он от бешенства горя инмом. вед от


У такого подхода к генерации есть недостаток: часто будут генерироваться токены, соврешенно не подходящие по смыслу, потому что иногда сэмплирование будет происходить из токенов с маленькой вероятностью. При этом чем длиннее предложение, тем больше вероятность, что такое случится. В следующем задании вам нужно будет реализовать более продвинутые способы генерации токенов.

# Задание 1. Температура, top-k, nucleus sampling.

Есть несколько способов улучшить качество генерации текстов из языковых моделей.
Первый - варьировние температуры. Логиты (выходы нашей сети) делятся на число $T$ - температуру. Она регулирует энтропию в распределении. При $T = 1$ получается сэмплирование из распределения, которое выдаёт язмодель. Елсли $T > 1$, то распределение становится ближе к равномерному, и соответсвенно появляется больше разнообразия, но и выходы становятся более хаотичными. Если наоборот делать $T$ меньше $1$, то полученное распределение будет приближаться к вырожденному распределению с вероятносстью $1$ у самого вероятного токена. Иначе говоря, при маленьких $T$ получается почти argmax-сэмплирование.

### 1.1 Добавьте температуру к выходам сети, попробуйте значения больше и меньше $1$. Выберите, какое значение на ваш взгляд даёт наиболее красивые результаты.

In [ ]:
def sample_generate(model, ids, max_len, EOS_id, T):
    for j in range(len(ids), max_len):
      x = torch.tensor(ids).unsqueeze(0).to(device)

      x_len = x.size(1)

      x = model.pos_emb(model.emb(x))

      attn_mask = nn.Transformer.generate_square_subsequent_mask(None, x_len).to(device)

      out = model.layers(x.transpose(0, 1), attn_mask).transpose(0, 1)

      out = model.out(out)

      dist = torch.distributions.categorical.Categorical(logits=out[0][x_len-1] / T)

      next_id = dist.sample().item()

      if next_id == EOS_id:
        break

      ids.append(next_id)

    return ids

In [ ]:
model.eval()

start_ids = [SOS_id]

sample_ids = sample_generate(model, start_ids, 64, EOS_id, 0.9)

sent = tokenizer.decode(sample_ids[1:])

print(sent)

скоро на мгновенье, начасов с новой далзал на воз!"
тяжелые – в кровитаны про борь, ничегонюю сом,
засил мне улыбка в дольний поэту светлогоуй,
да мир, старый прият сокину на скатер,
бить за врагом



Избежать появления маловероятных токенов можно напрямую. На каждом шаге можно рассматривать только $k$ токенов, имеющих максимальную вероятность, где $k$ - гиперпараметр.

### 1.2 Реализуйте top-k сэмплирование. Попробуйте разные значения $k$. Выберите наилучшее.


In [ ]:
def sample_generate(model, ids, max_len, EOS_id, k):
    for j in range(len(ids), max_len):
      x = torch.tensor(ids).unsqueeze(0).to(device)

      x_len = x.size(1)

      x = model.pos_emb(model.emb(x))

      attn_mask = nn.Transformer.generate_square_subsequent_mask(None, x_len).to(device)

      out = model.layers(x.transpose(0, 1), attn_mask).transpose(0, 1)

      out = model.out(out)

      topv, topi = out[0][-1].topk(k)

      dist = torch.distributions.categorical.Categorical(logits=topv)

      next_id = topi[dist.sample().item()].item()

      if next_id == EOS_id:
        break

      ids.append(next_id)

    return ids

In [ ]:
model.eval()

start_ids = [SOS_id]

sample_ids = sample_generate(model, start_ids, 64, EOS_id, 64)

sent = tokenizer.decode(sample_ids[1:])

print(sent)

о, ты, я твоею, мой конь с моряок,
из своего без душистывай.
от за той бесляет отцал возал...
уй мне отдать ты в поле,
молодец, кто голос твой зри.
ты с мечтой в граждан,
ты – бесстыд


Рассматривать $k$ наиболее вероятных токенов на каждом шаге может быть неоптимально: реальное количество подходящих токенов может быть разным от шага к шагу. В статье https://arxiv.org/pdf/1904.09751.pdf предложили альтернативу: оставлять на каждом шаге токены, чья вероятность в сумме даёт фиксированное значение $p$, и производить сэмплирование из них.

### 1.3 Реализуйте nucleus sampling. Попробуйте разные значения $p$. Покажите, как меняется число токенов, в сумме составляющих данную вероятность. Выерите лучшее $p$.

# Задание 2. Генерация русских фамилий

Теперь ваша задача - сгенерировать данные другой природы. Вместо стихов нужно сгенерировать русские фамилии.

### 1.1 Скачайте датасет https://mydata.biz/storage/download/ebcdfe6fb2d546398010e0d6564a79bb/names.zip. Он содержит список русских имён и фамилий в формате csv, нас будут интересовать фамилии. Обработайте данные.

### 1.2 Создайте словарь токенов, подходящий для задачи.

### 1.3 Обучите модель, сгенерируйте несколько новых примеров, оцените их качество.